In [1]:
import weaviate
import os
import json

wv_url = "https://some-endpoint.weaviate.network"
api_key = os.environ.get("OPENAI_API_KEY")

auth = weaviate.AuthClientPassword(
    username=os.environ.get("WCS_USER"),
    password=os.environ.get("WCS_PASS"),
)

client = weaviate.Client(
    url=wv_url,
    auth_client_secret=auth,
    additional_headers={"X-OpenAI-Api-Key": api_key},
)

In [2]:
# client.schema.delete_all()

In [3]:
import wv_datasets

dataset = wv_datasets.WineReviews()  # Try JeopardyQuestions, WikiArticles, or WineReviews
print(dataset.classes_in_schema(client))
class_defs = dataset.see_class_definitions()
print(json.dumps(class_defs, indent=2))

{'WineReview': False}
[
  {
    "class": "WineReview",
    "vectorizer": "text2vec-openai",
    "properties": [
      {
        "name": "review_body",
        "dataType": [
          "text"
        ],
        "description": "Review body"
      },
      {
        "name": "title",
        "dataType": [
          "string"
        ],
        "description": "Name of the wine"
      },
      {
        "name": "country",
        "dataType": [
          "string"
        ],
        "description": "Originating country"
      },
      {
        "name": "points",
        "dataType": [
          "int"
        ],
        "description": "Review score in points"
      },
      {
        "name": "price",
        "dataType": [
          "number"
        ],
        "description": "Listed price"
      }
    ]
  }
]


In [4]:
dataset.add_to_schema(client)
dataset.upload_objects(client, batch_size=100)

50it [00:00, 3094.61it/s]


True

In [5]:
client.query.get(dataset.get_class_names()[0], ["title"]).with_limit(5).do()

{'data': {'Get': {'WineReview': [{'title': 'White Oak 2009 Cabernet Sauvignon (Napa Valley)'},
    {'title': 'Domaine Foretal 2015  Beaujolais-Villages'},
    {'title': 'Château Rauzan-Ségla 2012  Margaux'},
    {'title': 'Clos La Chance 2008 Whitestone Vineyard Cabernet Sauvignon (Central Coast)'},
    {'title': 'Luigi Bosca 2005 D.O.C Single Vineyard Malbec (Luján de Cuyo)'}]}}}

In [6]:
client.query.aggregate(dataset.get_class_names()[0]).with_meta_count().do()

{'data': {'Aggregate': {'WineReview': [{'meta': {'count': 50}}]}}}

Exception in thread TokenRefresh:
Traceback (most recent call last):
  File "/Users/jphwang/mambaforge/lib/python3.10/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/Users/jphwang/mambaforge/lib/python3.10/site-packages/urllib3/util/connection.py", line 72, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/Users/jphwang/mambaforge/lib/python3.10/socket.py", line 955, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 8] nodename nor servname provided, or not known

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/jphwang/mambaforge/lib/python3.10/site-packages/urllib3/connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "/Users/jphwang/mambaforge/lib/python3.10/site-packages/urllib3/connectionpool.py"